In [1]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
from src.main import*
import time as tm
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
formula = dict(main = 'y ~ Binomial(total_count = density, logits = p)',
               likelihood = 'p ~ alpha[tank]', 
               prior = 'alpha ~ Normal(a_bar, sigma)',
               prior1 = 'a_bar ~ Normal(0.,1.5)',
               prior2 = 'sigma ~ Exponential(1)'
               )

start = tm.time()   
m13_2 = model(formula, d, float=32)
#m13_2.fit(observed_data = dict(y =d.surv.astype('float32').values), num_chains= 4, num_results=2000)
#end = tm.time()    
#print(f"BI took: {end - start:.4f} seconds")

2024-03-20 10:00:50.469742: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-20 10:00:50.496721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 10:00:50.496743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 10:00:50.497353: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 10:00:50.502849: I tensorflow/core/platform/cpu_feature_guar

In [1]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
from src.main_jax import*
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
formula = dict(main = 'y ~ Binomial(total_count = density, logits = p)',
               likelihood = 'p ~ alpha[tank]', 
               prior = 'alpha ~ Normal(a_bar, sigma)',
               prior1 = 'a_bar ~ Normal(0.,1.5)',
               prior2 = 'sigma ~ Exponential(1)'
               )

m13_2 = model(formula, d, float=32)
m13_2.tensor_seq

2024-03-20 08:23:40.304893: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 08:23:40.304943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 08:23:40.305543: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 08:23:40.786388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


jax.local_device_count  32


<tfp.distributions.JointDistributionSequentialAutoBatched 'JointDistributionSequentialAutoBatched' batch_shape=[] event_shape=[[1], [1], [1, 48], [48]] dtype=[float32, float32, float32, float32]>

In [ ]:
import jax.numpy as jnp
@jit
def replace_elements(list_of_arrays, index,  replace_array):
    # Ensure replace_array is a JAX array
    #replace_array = jnp.array(replace_array)

    # Create a new list with replaced elements
    #new_list = [jnp.where(jnp.isclose(i, index), replace_array, arr)
    #            for i, arr in enumerate(list_of_arrays)]
    new_list = jnp.empty(len(list_of_arrays))
    for i, arr in enumerate(list_of_arrays):
        print(i)
        print(jnp.all(jnp.isclose(i, index)))
        jnp.where(jnp.isclose(i, index), )
        if jnp.all(jnp.isclose(i, index)):
            print('replace')
            new_list[i] =  replace_array
        else:
            new_list[i] =  list_of_arrays[i]

    return new_list
replace_elements(init_params, int(2), init_params[0])

In [2]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
from src.main import*
import time as tm

import os
import warnings

import arviz as az
import matplotlib.pyplot as plt
import pandas as pd

import jax.numpy as jnp
from jax import lax, random
from jax.scipy.special import expit

import numpyro
import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size
from numpyro.infer import MCMC, NUTS, Predictive

az.style.use("arviz-darkgrid")
numpyro.set_platform("cpu")
numpyro.set_host_device_count(4)

reedfrogs = pd.read_csv("../data/reedfrogs.csv", sep=";")
d = reedfrogs
d["tank"] = jnp.arange(d.shape[0])

dat = dict(S=d.surv.values, N=d.density.values, tank=d.tank.values)

formula = dict(main = 'y ~ Binomial(total_count = density, logits = p)',
               likelihood = 'p ~ alpha[tank]', 
               prior = 'alpha ~ Normal(a_bar, sigma)',
               prior1 = 'a_bar ~ Normal(0.,1.5)',
               prior2 = 'sigma ~ Exponential(1)'
               )

def model(tank, N, S):
    a_bar = numpyro.sample("a_bar", dist.Normal(0, 1.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape=tank.shape)
    logit_p = a[tank]
    numpyro.sample("S", dist.Binomial(N, logits=logit_p), obs=S)

start = tm.time()  
m13_2 = MCMC(NUTS(model), num_warmup=500, num_samples=2000, num_chains=4)
m13_2.run(random.PRNGKey(0), **dat)
end = tm.time()    
print(f"NumpyPro took: {end - start:.4f} seconds")

/home/sosa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
Compiling.. :   0%|          | 0/2500 [00:00<?, ?it/s]








Running chain 3: 100%|██████████| 2500/2500 [00:01<00:00, 1355.74it/s]

NumpyPro took: 2.0605 seconds


# Dict model

In [13]:
import jax
import jax.numpy as jnp
from jax import random, jit
from tensorflow_probability.substrates import jax as tfp
from tensorflow_probability.substrates.jax.distributions import JointDistributionNamedAutoBatched as JDNAB
tfd = tfp.distributions
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
# Define the model using JointDistributionNamed
m = {}
m['alpha'] = lambda a_bar, sigma: tfd.Sample(tfd.Normal(a_bar, sigma), sample_shape=48)
m['a_bar'] = tfd.Normal(0., 1.5, name='a_bar')
m['sigma'] = tfd.Exponential(1., name='sigma')
m['y'] = lambda alpha: tfd.Independent(
    tfd.Binomial(
        total_count=jnp.array(d.density.values, dtype=jnp.float32),
        logits=jnp.squeeze(jnp.take(alpha, jnp.array(jnp.array(d.tank.values, dtype=jnp.float32), dtype=jnp.int32), axis=-1)),
        name='main'), reinterpreted_batch_ndims=1)

dist = JDNAB(m)

observed_data = d.surv.astype('float32').values
observed_data_jax = jnp.array(observed_data)  # Convert observed_data to JAX array

n_chain = jax.local_device_count()
init_key, sample_key = random.split(random.PRNGKey(0))
init_params = dist.sample(seed=jnp.array(init_key, dtype=jnp.uint32))
init_params.pop('y')
init_params = list(init_params.values())
init_params

def target_log_prob(*params):
    param_dict = {}
    keys = dist._flat_resolve_names()
    for i, key in enumerate(keys):
        if key != 'y':
            param_dict[key] = params[i]
    param_dict['y'] = observed_data_jax
    return dist.log_prob(param_dict)

@jit
def run_chain(key, obs_name = 'y'):
    init_key, sample_key = random.split(random.PRNGKey(0))
    init_params = dist.sample(seed=jnp.array(init_key, dtype=jnp.uint32))
    init_params.pop(obs_name)
    init_params = list(init_params.values())   

    kernel = tfp.mcmc.NoUTurnSampler(target_log_prob, 1e-3)
    return tfp.mcmc.sample_chain(2000,
                                  current_state=init_params,
                                  parallel_iterations = 4,
                                  kernel=kernel,
                                  trace_fn=lambda _, results: results.target_log_prob,
                                  num_burnin_steps=500,
                                  seed=key)

start = tm.time() 
states, log_probs = run_chain(sample_key)
end = tm.time()    
print(f"TF took: {end - start:.4f} seconds")

TF took: 3.3722 seconds


# Dict in functions

In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit
from tensorflow_probability.substrates import jax as tfp
from tensorflow_probability.substrates.jax.distributions import JointDistributionNamedAutoBatched as JDNAB
tfd = tfp.distributions
import random as r
from functools import partial


class test():
    def __init__(self, obs_names, observed_data_jax) :
        d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
        d["tank"] = np.arange(d.shape[0])
        # Define the model using JointDistributionNamed
        m = {}
        m['alpha'] = lambda a_bar, sigma: tfd.Sample(tfd.Normal(a_bar, sigma), sample_shape=48)
        m['a_bar'] = tfd.Normal(0., 1.5, name='a_bar')
        m['sigma'] = tfd.Exponential(1., name='sigma')
        m['y'] = lambda alpha: tfd.Independent(
            tfd.Binomial(
                total_count=jnp.array(d.density.values, dtype=jnp.float32),
                logits=jnp.squeeze(jnp.take(alpha, jnp.array(jnp.array(d.tank.values, dtype=jnp.float32), dtype=jnp.int32), axis=-1)),
                name='main'), reinterpreted_batch_ndims=1)

        self.tensor = JDNAB(m)
        self.obs_names = obs_names
        self.observed_data_jax = observed_data_jax
        self.keys = self.tensor._flat_resolve_names()

    @partial(jit, static_argnums=(0,))
    def target_log_prob(self, *params):
        param_dict = {}
        for i, key in enumerate(self.keys):
            if key != self.obs_names:
                param_dict[key] = params[i]
        param_dict[self.obs_names] = self.observed_data_jax
        return self.tensor.log_prob(param_dict)

    
    @partial(jit, static_argnums=(0,))
    def run_chain(self, key):        
        init_key, sample_key = random.split(random.PRNGKey(0))
        init_params = self.tensor.sample(seed=jnp.array(init_key, dtype=jnp.uint32))
        init_params.pop(self.obs_names)
        init_params = list(init_params.values())   
        kernel = tfp.mcmc.NoUTurnSampler(target_log_prob_fn = self.target_log_prob, step_size = 1e-3)
        return tfp.mcmc.sample_chain(2000,
                                      current_state=init_params,
                                      parallel_iterations = 4,
                                      kernel=kernel,
                                      trace_fn=lambda _, results: results.target_log_prob,
                                      num_burnin_steps=500,
                                      seed=key)
    
    @jit
    def parallel_keys(self, n_chains):
        return jax.random.split(random.PRNGKey(0), n_chain)
    
    @jit
    def fit(self, n_chain = 4):
        #rng_keys = jax.random.split(random.PRNGKey(0), n_chain)
        return jax.pmap(self.run_chain)(self.parallel_keys(int(n_chain)))

observed_data = d.surv.astype('float32').values
observed_data_jax = jnp.array(observed_data)  # Convert observed_data to JAX array
m = test(obs_names = 'y', observed_data_jax = observed_data_jax)

init_key, sample_key = random.split(random.PRNGKey(0))
start = tm.time() 
res = m.fit(n_chain = 4)
end = tm.time()    
print(f"TF took: {end - start:.4f} seconds")